# Neural network Tuning

In [1]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Importing necesary packages
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV 
from keras.models import Sequential
from keras.layers import Dense
import warnings
warnings.filterwarnings('ignore')
from keras.wrappers.scikit_learn import KerasClassifier

2023-09-11 11:08:15.565966: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'keras.wrappers'

# part 1 - Tuning batch and epoch

In [2]:
# load the dataset for India diabetes
data = pd.read_csv('diabetes.csv')

In [3]:
#viewing data
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [5]:
data.shape

(768, 9)

In [6]:
data.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [7]:
print( data['Glucose'].value_counts()[0] )
print( data['BloodPressure'].value_counts()[0] )
print( data['SkinThickness'].value_counts()[0] )
print( data['Insulin'].value_counts()[0] )
print( data['BMI'].value_counts()[0] )

5
35
227
374
11


In [8]:
data = data.drop(data[(data['Glucose'] == 0) | (data['BloodPressure'] == 0) | (data['BMI'] == 0)].index)
data = data.reset_index()

In [9]:
data.shape

(724, 10)

In [10]:
data.SkinThickness.unique()

array([35, 29,  0, 23, 32, 45, 19, 47, 38, 30, 41, 33, 26, 15, 36, 11, 31,
       37, 42, 25, 18, 24, 39, 27, 21, 34, 10, 60, 13, 20, 22, 28, 54, 40,
       51, 56, 14, 17, 50, 44, 12, 46, 16,  7, 52, 43, 48,  8, 49, 63, 99])

In [17]:
for i in range(len(data)):
    if data['SkinThickness'][i] == 0 :
        age_group = data[data['Age'] == data['Age'][i]]
        if data['SkinThickness'].mode().iloc[0] == 0 :
            i = len(age_group['SkinThickness'].mode())
            if i == 1 :
                pass
            else:
                data['SkinThickness'][i] = age_group['SkinThickness'].mode().iloc[1]
        else:
            data['SkinThickness'][i] = age_group['SkinThickness'].mode().iloc[0]
    else:
        pass

In [18]:
print( data['SkinThickness'].value_counts()[0] )

190


In [13]:
for i in range(len(data)):
    if data['Insulin'][i] == 0 :
        age_group = data[data['Age'] == data['Age'][i]]
        if data['Insulin'].mode().iloc[0] == 0 :
            i = len(age_group['Insulin'].mode())
            if i == 1 :
                if data['Insulin'].median() == 0:
                    pass
                else:
                    data['Insulin'][i] = age_group['Insulin'].median()
            else:
                data['Insulin'][i] = age_group['Insulin'].mode().iloc[1]
        else:
            data['Insulin'][i] = age_group['Insulin'].mode().iloc[0]
    else:
        pass

In [14]:
print( data['Insulin'].value_counts()[0] )

330


In [22]:
# split into predictors and response
X = data.iloc[:,0:8] #predictors
Y = data.iloc[:,8] #response

In [23]:
# Function to create model,for KerasClassifier
def create_my_model():
    #defining my model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    mymodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return mymodel

In [24]:
# create model
model = KerasClassifier(build_fn=create_my_model)

NameError: name 'KerasClassifier' is not defined

In [25]:
# define the grid search parameters
batchSize = [10, 20, 40, 60, 80, 100]
epochs = [10, 30, 50]

In [36]:
parameter_grid = dict(batch_size=batchSize, epochs=epochs)


mygrid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = mygrid.fit(X, Y)

Epoch 1/50
768/768 [==============================] - 1s 1ms/step - loss: 12.3063 - accuracy: 0.4401
Epoch 2/50
768/768 [==============================] - 0s 259us/step - loss: 3.1792 - accuracy: 0.5951
Epoch 3/50
768/768 [==============================] - 0s 240us/step - loss: 2.3888 - accuracy: 0.6120
Epoch 4/50
768/768 [==============================] - 0s 280us/step - loss: 2.0140 - accuracy: 0.6029
Epoch 5/50
768/768 [==============================] - 0s 263us/step - loss: 1.5988 - accuracy: 0.6185
Epoch 6/50
768/768 [==============================] - 0s 250us/step - loss: 1.3167 - accuracy: 0.6276
Epoch 7/50
768/768 [==============================] - 0s 406us/step - loss: 1.1392 - accuracy: 0.6081
Epoch 8/50
768/768 [==============================] - 0s 362us/step - loss: 1.0216 - accuracy: 0.6094
Epoch 9/50
768/768 [==============================] - 0s 340us/step - loss: 0.9658 - accuracy: 0.6081
Epoch 10/50
768/768 [==============================] - 0s 340us/step - loss: 0.9297

In [37]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.679688 using {'batch_size': 10, 'epochs': 50}


# part 2- Optimizing Activation Function

In [65]:
# Function to create model
def create_my_model(optimizer='adam'):
    # create model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
    # Compile model
    mymodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return mymodel

In [66]:
# create model
model = KerasClassifier(build_fn=create_my_model, epochs=50, batch_size=10)

In [67]:
# define the grid search parameters
optimizer = ['SGD','Adadelta', 'RMSprop', 'Adagrad','Adam']
parameter_grid = dict(optimizer=optimizer)

In [68]:
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

Epoch 1/50
768/768 [==============================] - 1s 677us/step - loss: 1.5185 - accuracy: 0.6445
Epoch 2/50
768/768 [==============================] - 0s 177us/step - loss: 0.6720 - accuracy: 0.6615
Epoch 3/50
768/768 [==============================] - 0s 177us/step - loss: 0.6598 - accuracy: 0.6602
Epoch 4/50
768/768 [==============================] - 0s 198us/step - loss: 0.6476 - accuracy: 0.6602
Epoch 5/50
768/768 [==============================] - 0s 177us/step - loss: 0.6397 - accuracy: 0.6849
Epoch 6/50
768/768 [==============================] - 0s 177us/step - loss: 0.6369 - accuracy: 0.6732
Epoch 7/50
768/768 [==============================] - 0s 229us/step - loss: 0.6322 - accuracy: 0.6745
Epoch 8/50
768/768 [==============================] - 0s 208us/step - loss: 0.6273 - accuracy: 0.6810
Epoch 9/50
768/768 [==============================] - 0s 187us/step - loss: 0.6226 - accuracy: 0.6784
Epoch 10/50
768/768 [==============================] - 0s 177us/step - loss: 0.621

In [69]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.680990 using {'optimizer': 'SGD'}


# How can you improve accuracy of the model? 